<a href="https://colab.research.google.com/github/raki-rankawat/stm32/blob/main/CIFAR10_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Trained Model

In [12]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [13]:
# Data Loaders
batch_size = 64

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [14]:
# CNN Model
class CIFARConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.fc1 = nn.Linear(128 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 10)

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2) # 32 -> 16

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) # 16 -> 8

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2) # 8 -> 4

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) # 4 -> 2

        x = x.view(x.size(0), -1) # Flatten

        # x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [15]:
# Random Seeds and Model Instance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(41)
model = CIFARConvNet().to(device)

In [16]:
# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
# Training
def train(model, loader, criterion, optimizer, device):
    model.train() # training mode

    correct = 0
    total = 0
    running_loss = 0

    for X, y in loader:
        X = X.to(device)
        y = y.to(device)

        # Forward
        outputs = model(X)

        # Loss
        loss = criterion(outputs, y)

        # Backward
        optimizer.zero_grad(set_to_none = True)
        loss.backward()
        optimizer.step()

        # Statistics
        batch_size = y.size(0)
        running_loss += loss.item() * batch_size
        preds = outputs.argmax(dim = 1)
        correct += (preds == y).sum().item()
        total += batch_size

    avg_loss = running_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

In [18]:
# Testing
def test(model, loader, criterion):
  model.eval() # testing mode

  correct = 0
  total = 0
  running_loss = 0

  with torch.no_grad():
    for X, y in loader:
      X = X.to(device)
      y = y.to(device)

      # Forward
      outputs = model(X)

      # Loss
      loss = criterion(outputs, y)

      # Statistics
      batch_size = y.size(0)
      running_loss += loss.item() * batch_size
      preds = outputs.argmax(dim = 1)
      correct += (preds == y).sum().item()
      total += batch_size

  avg_loss = running_loss / total
  accuracy = correct / total

  return avg_loss, accuracy

In [19]:
epochs = 10
start_time = time.time()

for epoch in range(1, epochs + 1):
  train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
  test_loss, test_acc = test(model, test_loader, criterion)

  print(
      f"Epoch: {epoch}/{epochs} | "
      f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc * 100:.2f}% | "
      f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc * 100:.2f}%"
  )

print(f"Training Time: {(time.time() - start_time) / 60} minutes!")

Epoch: 1/10 | Train Loss: 1.4135 | Train Acc: 48.45% | Test Loss: 1.1073 | Test Acc: 60.99%
Epoch: 2/10 | Train Loss: 1.0686 | Train Acc: 62.03% | Test Loss: 1.0005 | Test Acc: 64.77%
Epoch: 3/10 | Train Loss: 0.9478 | Train Acc: 66.56% | Test Loss: 0.8261 | Test Acc: 71.20%
Epoch: 4/10 | Train Loss: 0.8748 | Train Acc: 69.27% | Test Loss: 0.7996 | Test Acc: 72.43%
Epoch: 5/10 | Train Loss: 0.8259 | Train Acc: 71.04% | Test Loss: 0.7552 | Test Acc: 73.19%
Epoch: 6/10 | Train Loss: 0.7817 | Train Acc: 72.76% | Test Loss: 0.7158 | Test Acc: 74.85%
Epoch: 7/10 | Train Loss: 0.7584 | Train Acc: 73.64% | Test Loss: 0.6881 | Test Acc: 75.87%
Epoch: 8/10 | Train Loss: 0.7299 | Train Acc: 74.50% | Test Loss: 0.6995 | Test Acc: 75.83%
Epoch: 9/10 | Train Loss: 0.7038 | Train Acc: 75.47% | Test Loss: 0.6851 | Test Acc: 76.99%
Epoch: 10/10 | Train Loss: 0.6925 | Train Acc: 75.88% | Test Loss: 0.6301 | Test Acc: 78.25%
Training Time: 6.782521299521128 minutes!


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
torch.save(model.state_dict(), "/content/drive/My Drive/Colab Notebooks/stm_cifar10_model.pth")
print("✅ Model saved as stm_cifar10_model.pth")

✅ Model saved as stm_cifar10_model.pth
